# Perceptron algorithm and its variants

#### Define library and constants.

In [46]:
import numpy as np

number_of_features = 0
number_of_classes = 0
dataset_size = 0

seed_val = 107
max_itr = 1000

np.random.seed(seed_val)

#### Define a class that stands for a class. It has a name and an array to hold the feature vectors.

In [47]:
class Object:
    def __init__(self, class_name):
        self.class_name = class_name
        self.features = []


Object_Dictionary = {}

#### read the dataset

In [48]:
def read_dataset(train_file):
    global number_of_features, number_of_classes, dataset_size, Object_Dictionary

    f = open(train_file, "r")
    lines = f.readlines()

    number_of_features, number_of_classes, dataset_size = map(int, lines[0].rstrip().split())

    for i in range(dataset_size):
        data = lines[i + 1].rstrip().split()
        class_name = int(data[number_of_features])

        if class_name not in Object_Dictionary:
            Object_Dictionary[class_name] = Object(class_name)

        Object_Dictionary[class_name].features.append(np.array(data[: number_of_features], dtype=float))

## Basic perceptron algorithm


In [49]:
class Perceptron:
    def __init__(self):
        self.learning_rate = 0.1
        self.w = np.random.uniform(-1, 1, number_of_features + 1)
        
    
    # train the model
    def train_model(self):
        for itr in range(max_itr):
            misclassified = []

            for key in Object_Dictionary.keys():
                for i in range(len(Object_Dictionary[key].features)):
                    x = Object_Dictionary[key].features[i]
                    x = np.append(x, 1)
                    x = np.array(x)

                    val = np.dot(self.w, x)

                    # actually omega1, classified as omega2
                    if key == 1 and val < 0:
                        misclassified.append(x * -1)

                    # actually omega2, classified as omega1
                    elif key == 2 and val > 0:
                        misclassified.append(x)

            if len(misclassified) == 0:
                print("training done in", itr, "th iteration")
                break

            summation = np.zeros(number_of_features + 1)
            for i in range(len(misclassified)):
                summation += misclassified[i]

            summation = self.learning_rate * summation
            self.w = self.w - summation
            
    
    # test the model
    def test_model(self, test_file):
        correctly_detected = 0
        
        result_output = open("results.txt", "w")
        result_output.write("Perceptron Alogrithm:\n\n")
        
        f = open(test_file, "r")
        lines = f.readlines()

        for i in range(dataset_size):
            data = list(map(float, lines[i].rstrip().split()))

            actual_class = int(data[number_of_features])
            data[number_of_features] = 1
            x = np.array(data)

            prod = np.dot(self.w, x)
            if prod >= 0:
                predicted_class = 1
            else:
                predicted_class = 2

            if predicted_class == actual_class:
                correctly_detected += 1
            else:
                s = "Sample no: " + str(i + 1) + ". "
                s += "Features: " + str(data[:number_of_features]) + ". "
                s += "Actual class: " + str(actual_class) + ". "
                s += "Predicted class: " + str(predicted_class) + "\n"
                result_output.write(s)

        accuracy = (correctly_detected / dataset_size) * 100
        print("accuracy of basic perceptron algorithm:", accuracy, "%")
        
        # free the dictionary
        Object_Dictionary.clear()
        
        

## Pocket algorithm

In [50]:
class Pocket_Perceptron:
    def __init__(self):
        self.learning_rate = 0.1
        self.w = np.random.uniform(-1, 1, number_of_features + 1)
        self.ws = self.w
        self.hs = 0
        
    
    # train the model
    def train_model(self):
        for itr in range(max_itr):
            misclassified = []

            for key in Object_Dictionary.keys():
                for i in range(len(Object_Dictionary[key].features)):
                    x = Object_Dictionary[key].features[i]
                    x = np.append(x, 1)
                    x = np.array(x)

                    val = np.dot(self.w, x)

                    # actually omega1, classified as omega2
                    if key == 1 and val < 0:
                        misclassified.append(x * -1)

                    # actually omega2, classified as omega1
                    elif key == 2 and val > 0:
                        misclassified.append(x)
            
            # w gives less misclassified so update
            if self.hs < dataset_size - len(misclassified):
                self.hs = dataset_size - len(misclassified)
                self.ws = self.w
            
            if len(misclassified) == 0:
                print("training done in", itr, "th iteration")
                break

            summation = np.zeros(number_of_features + 1)
            for i in range(len(misclassified)):
                summation += misclassified[i]

            summation = self.learning_rate * summation
            self.w = self.w - summation
            
    
    # test the model
    def test_model(self, test_file):
        correctly_detected = 0
        
        result_output = open("results.txt", "w")
        result_output.write("Perceptron Alogrithm:\n\n")
        
        f = open(test_file, "r")
        lines = f.readlines()

        for i in range(dataset_size):
            data = list(map(float, lines[i].rstrip().split()))

            actual_class = int(data[number_of_features])
            data[number_of_features] = 1
            x = np.array(data)

            prod = np.dot(self.ws, x)
            if prod >= 0:
                predicted_class = 1
            else:
                predicted_class = 2

            if predicted_class == actual_class:
                correctly_detected += 1
            else:
                s = "Sample no: " + str(i + 1) + ". "
                s += "Features: " + str(data[:number_of_features]) + ". "
                s += "Actual class: " + str(actual_class) + ". "
                s += "Predicted class: " + str(predicted_class) + "\n"
                result_output.write(s)

        accuracy = (correctly_detected / dataset_size) * 100
        print("accuracy of basic perceptron algorithm:", accuracy, "%")
        
        # free the dictionary
        Object_Dictionary.clear()
        
        

#### run perceptron algorithm

In [51]:
test_file_1 = "./dataset/testLinearlySeparable.txt"
train_file_1 = "./dataset/trainLinearlySeparable.txt"

test_file_2 = "./dataset/testLinearlyNonSeparable.txt"
train_file_2 = "./dataset/trainLinearlyNonSeparable.txt"

read_dataset(train_file_2)
p = Perceptron()
p.train_model()
p.test_model(test_file_2)

accuracy of basic perceptron algorithm: 97.5 %


#### run pocket algorithm

In [52]:
test_file_1 = "./dataset/testLinearlySeparable.txt"
train_file_1 = "./dataset/trainLinearlySeparable.txt"

test_file_2 = "./dataset/testLinearlyNonSeparable.txt"
train_file_2 = "./dataset/trainLinearlyNonSeparable.txt"

read_dataset(train_file_2)
p = Pocket_Perceptron()
p.train_model()
p.test_model(test_file_2)

accuracy of basic perceptron algorithm: 97.75 %
